In [1]:
!pip install pymystem3==0.1.10

  Attempting uninstall: pymystem3
    Found existing installation: pymystem3 0.2.0
    Uninstalling pymystem3-0.2.0:
      Successfully uninstalled pymystem3-0.2.0


In [2]:
from pymystem3 import Mystem
mystem = Mystem() 

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [3]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter


In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords", quiet=True)
russian_stopwords = stopwords.words('russian')
russian_stopwords.append('дак')

In [5]:
url = 'https://borbushino.vogu35.ru/?text=%d1%80%d0%b0%d0%bd%d1%8c%d1%88%d0%b5-%d0%b8-%d1%82%d0%b5%d0%bf%d0%b5%d1%80%d1%8c'
page = rq.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [6]:
# Создание датасета, в который будут сохраняться данные с сайта.
text_and_its_passport={'Название':[], 'Тематические разделы':[], 'ФИО информанта':'', 'Год рождения (возраст)':'', 'Пол информанта':'', 'Уровень образования':'', 'Место записи': '', 'Год записи': '', 'Собиратель (ФИО)': '', 'Текст': ''}
Table_summary=pd.DataFrame.from_dict(text_and_its_passport, orient="index")
Table_summary=Table_summary.transpose()

In [ ]:
# Парсинг сайта.
while url != None:
  page = rq.get(url) #получаем информацию со страницы

  soup = BeautifulSoup(page.text, 'html.parser') #конвертируем информацию в объект BeautifulSoup

  text_and_its_passport={'Название':[], 'Тематические разделы':[], 'ФИО информанта':'', 'Год рождения (возраст)':'', 'Пол информанта':'', 'Уровень образования':'', 'Место записи': '', 'Год записи': '', 'Собиратель (ФИО)': '', 'Текст': ''}
  # создаем словарь для информации со страницы (ключи - параметры описания текста, значения - значения этих параметров)

  text_and_its_passport['Название']=soup.find('h1').get_text() #Извлекаем название текста и добавляем его в словарь

  fragment=soup.strong.parent.parent #Достаем из объекта BeautifulSoup блок DIV, содержащий нужную нам информацию.

  while fragment.div.i != None: #Ищем и убираем весь курсивчик (=реплики собирателей).
    fragment.div.i.extract()

  text_and_its_passport['Текст']=fragment.div.get_text() #Извлекаем текст и добавляем его в словарь
  fragment.div.extract() #Удаляем текст из блока DIV, чтобы не мешал в дальнейшей обработке.

  def parse_tags (frag): #Эта функция достаёт из тега <P> параметр описания текста и его значение / значения.
    names=[]
    for string in frag.stripped_strings:
      if string !=',' and string !='.':
        names.append(string)
    return(names)

  fragment_2 = fragment.p #Выбираем первый тег <P>

  while fragment_2 != '\n': 
    result = parse_tags(fragment_2) #Выбираем тег <P> в функцию parse_tags
    if result[0][:-1] in text_and_its_passport:
      text_and_its_passport[result[0][:-1]]=result[1:] #Записываем полученную от функции информацию в словарь
    fragment_2 = fragment_2.next_sibling.next_sibling #Переходим к следующему тегу <P>
  Table=pd.DataFrame.from_dict(text_and_its_passport, orient="index") #Записываем информацию из словаря в датасет.
  Table=Table.transpose()
  Table_summary=Table_summary.append(Table)
  url = soup.find('div', "nav-next")
  if url != None:
    url = url.a.get('href')
    print(url)
Table_summary.to_csv('Table_summary.csv')

In [8]:
#Добавление в датасет столбца с числовыми индексами строк.
i=0
index = []
while i <= Table_summary.shape[0] - 1:
  index.append(i)
  i=i+1
Table_summary['index'] =  index
Table_summary=Table_summary.set_index('index')

In [9]:
# Препроцессинг.
def preprocessing (i):
    i=i.lower()
    i=re.sub('тексты публикуются с сохранением авторской грамматики, орфографии, пунктуации и графики.', '', i)
    i=re.sub('\[.*\]', '', i)#Убираем транскрибированный текст.
    # i=re.sub('\[.*', '', i)#Убираем транскрибированный текст.
    # i=re.sub('.*\]',  '', i)#Убираем транскрибированный текст.
    # i=re.sub('\n\[.*\n.*\]\n',  '', i)
    i=re.sub('́', '', i)#Убираем ударения.
    i=re.sub('\n.+\]\n',  '', i)
    i=re.sub('\n\[.+', '', i)
    i=re.sub('\n.+//',  '', i)  
    i=re.sub('́', '', i)#Убираем ударения.
    i=re.sub('́',  '', i)#Убираем ударения.
    i=re.sub('\*',  '', i)#Убираем звездочки.
    i=re.sub('\n\xa0\n.*/', '', i)
    i=re.sub('ɣ',  'г', i)  
    i=re.sub('вообшэ',  'вообще', i)
    i=re.sub('\sшо[\s|,|\.|!|?|)]',  ' что ', i)  
    i=re.sub('\sчо[\s|,|\.|!|?|)]', 'что ', i)      
    i=re.sub('\sцо[\s|,|\.|!|?|)]',  ' что ', i)
    i=re.sub('\sчо-то[\s|,|\.|!|?|)]',  ' что-то ', i)
    i=re.sub('\sцо-то[\s|,|\.|!|?|)]',  ' что-то ', i)
    i=re.sub('\sчо-ни[\s|,|\.|!|?|)]',  ' что-ни ', i)
    i=re.sub('\sчо-нибудь[\s|,|\.|!|?|)]',  ' что-нибудь ', i)
    i=re.sub('\sжо[\s|,|\.|!|?|)]',  ' же ', i)
    i=re.sub('[\s]быу[\s|,|\.|!|?|)]',  ' был ', i)
    i=re.sub('[\s]уш[\s|,|\.|!|?|)]',  ' уж ', i)
    i=re.sub('крещоный',  'крещёный', i)
    i=re.sub('cмеется',  'смеется', i)
    i=re.sub('жы',  'жи', i)
    i=re.sub('шы',  'ши', i)
    i=re.sub('щы',  'щи', i)
    i=re.sub('коўда',  'когда', i)
    i=re.sub('раўно',  'равно', i)
    i=re.sub('ў',  'л', i)
    i=re.sub('ишо',  'ещё', i)
    i=re.sub('ищо',  'ещё', i)
    i=re.sub('ничё',  'ничего', i)
    i=re.sub('ничо',  'ничего', i)
    i=re.sub('жизь',  'жизнь', i)
    return (i)

In [10]:
#Делаем предобработка текстов в датасете.
Table_summary_without = Table_summary.copy(deep=True) #Копируем датасет.
texts = Table_summary_without["Текст"].to_list() #Извлекаем все тексты в список
texts_without = []#Убираем из каждого текст лишние элементы.
for text in texts:
  texts_without.append(preprocessing(text))
Table_summary_without['Текст_1'] =  texts_without
Table_summary_without = Table_summary_without.drop('Текст', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [11]:
#Функция для обработки нелемматизированных токенов.
def bastard_lemmatisation (bast_list):
  dict_bastards={}
  for i in bast_list:
    if 'ц' in i:
      experiment = re.sub('ц','ч', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if 'тца' in i:      
      experiment = re.sub('тца','тся', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if 'роб' in i:      
      experiment = re.sub('роб','раб', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if 'роб' in i:      
      experiment = re.sub('роб','реб', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if 'рост' in i:      
      experiment = re.sub('рост','раст', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if 'тци' in i:
      experiment = re.sub('тци','тся', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if i[-1]=='с':
      experiment = re.sub('с','сь', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if i[-2:]=='ся':
      experiment = re.sub('ся','сь', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment   
  for i in bast_list:
    if 'шн' in i:
      experiment = re.sub('шн','чн', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list: 
    if 'ца' in i:
      experiment = re.sub('ца','тся', i)
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if i[-1]=='у':
      experiment = i[:-1]+'л'    
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if i[-1]=='т':
      experiment = i+'ь'    
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  for i in bast_list:
    if i[-1]=='у':
      experiment = i[:-1]+'в'    
      exp = mystem.analyze(experiment)
      if exp[0]['analysis'] != []:
        if 'qual' not in exp[0]['analysis'][0]:
          dict_bastards[i]=experiment
  return dict_bastards

In [12]:
# Лемматизация.
def lemmatization (text):
  lemms=[]
  ms_analyzed = mystem.analyze(text)
  bastards = []
  for x in ms_analyzed:
    if len(x) == 2:
      if x['analysis'] != []:
        if 'qual' in x['analysis'][0]:
          bastards.append(x['text'])
  dict_bast=bastard_lemmatisation(bastards)
  for x in ms_analyzed:
    if x.get('analysis'):
      if x.get('analysis'):
        if 'qual' not in x['analysis'][0]:
          lemms.append(x['analysis'][0]['lex'])
        else:
          if x['text'] in dict_bast:
            analyze = mystem.analyze(x['text'])
            word = analyze[0]['analysis'][0]['lex']            
            lemms.append(word)
          else:
            lemms.append(x['analysis'][0]['lex'])
  return lemms

In [13]:
# Определение длины вектора.
def lenhgt_vector (vector):
    x=0
    for i in set(vector.keys()):
        x = x + (vector[i])**2
    x=x**0.5
    return x    

In [14]:
#Создание словаря текста (лемма - количество вхождений).
def dictionary (text):
    lemms_text=[]
    lemms_text_0 = lemmatization(text)
    for i in lemms_text_0:
      if i not in russian_stopwords:
        lemms_text.append(i)
    set_lemms_text = set(lemms_text)
    dict_text={}
    for i in set_lemms_text:
      dict_text[i] = lemms_text.count(i)
    return dict_text

In [15]:
#Вычисление косинусного сходства.
def cosine_distance (dict_1, dict_2):
    set_symmary = set(dict_1).union(set(dict_2))
    x_y = 0
    for i in set_symmary:
        if i in dict_1:
            x_1 = dict_1[i]
        else:
            x_1 = 0
        if i in dict_2:
            y_1 = dict_2[i]
        else:
            y_1 = 0
        x_y = x_y + (x_1*y_1)
    return x_y / (lenhgt_vector(dict_1)*lenhgt_vector(dict_2))

In [21]:
# Ввод текста.
My_text_1=input('Введите текст: ')
My_text_1=preprocessing(My_text_1)
My_text_1


Введите текст: Ве́чером пе́ред сно́м моли́лась, у́тром встава́ла моли́лась. Но она́ никогда́ не говори́ла: «Вну́ченька, вста́нь со мно́й, дава́й». Она́ мно́го зна́ла моли́тв, но я ни одно́й <смеётся>. Это че́стно говорю́. Я челове́к правосла́вный, но не фанати́чно ве́рующий. Фанати́зм я в любо́м слу́че не то́ко в э́том, но и в друго́м слу́че, я не признаю́! Фана́тики – это стра́шны лю́ди. Гм. Это ни к чему́ соверше́н. Ма́ма, ма́ма, я да́же не по́мню, шоб она́… Ну, иногда́ там перекре́стится, «сла́ва Бо́гу», садя́тся за сто́л, когда́ выхо́дят из стола́, благодаря́т Го́спода. Но на мое́й па́мяти, я не зна́ю, шоб ма́ма куда́-то ходи́ла в це́рковь. Когда́ це́рковь была́ закры́та, щас я вам год не скажу́, я уже́ то́же ста́ла забыва́ть. По бли́зости це́ркви не́ было. И наско́лько я по́мню, на большо́й террито́рии вот одна́ Покро́вская ц́ерковь, она так и не закрыва́лась. Как её откры́ли в девятна́дцатом ве́ке, так она́ и до си́х пор, до сего́дняшнего дня существу́ет, и рабо́тат и принима́ет 

'вечером перед сном молилась, утром вставала молилась. но она никогда не говорила: «внученька, встань со мной, давай». она много знала молитв, но я ни одной <смеётся>. это честно говорю. я человек православный, но не фанатично верующий. фанатизм я в любом случе не токо в этом, но и в другом случе, я не признаю! фанатики – это страшны люди. гм. это ни к чему совершен. мама, мама, я даже не помню, шоб она… ну, иногда там перекрестится, «слава богу», садятся за стол, когда выходят из стола, благодарят господа. но на моей памяти, я не знаю, шоб мама куда-то ходила в церковь. когда церковь была закрыта, щас я вам год не скажу, я уже тоже стала забывать. по близости церкви не было. и насколько я помню, на большой территории вот одна покровская церковь, она так и не закрывалась. как её открыли в девятнадцатом веке, так она и до сих пор, до сегодняшнего дня существует, и работат и принимает прихожан. где были ещё церкви? а потом, девочки милые, и не до церкви было. и не до обрядов этих было, п

In [17]:


# Создание словаря для введенного текста.
dict_My_text_1=dictionary(My_text_1)
result = []

# Вычисление косинусного сходства введенного текста с другими текстами датасета.
for Text_from_df in Table_summary_without['Текст_1']:
    dict_Text_from_df=dictionary(Text_from_df)
    r=cosine_distance (dict_My_text_1, dict_Text_from_df)
    r_1=round(r, 2)
    result.append(r_1)
# Добавление в датасет столбца с результатами вычисления косинусного сходства.
Table_summary_without['result'] =  result
# Сортировка датасета, определение 5 текстов с наибольшим косинусным сходством.
itog=Table_summary_without.sort_values(by='result', ascending=False)
data_frame_result=itog.head(5)
# Вывод результатов.
for i in data_frame_result.index:
  display(data_frame_result.loc[i]['result'])
  display(data_frame_result.loc[i]['ФИО информанта']) 
  display(data_frame_result.loc[i]['Текст_1'])

Введите текст: Ве́чером пе́ред сно́м моли́лась, у́тром встава́ла моли́лась. Но она́ никогда́ не говори́ла: «Вну́ченька, вста́нь со мно́й, дава́й». Она́ мно́го зна́ла моли́тв, но я ни одно́й <смеётся>. Это че́стно говорю́. Я челове́к правосла́вный, но не фанати́чно ве́рующий. Фанати́зм я в любо́м слу́че не то́ко в э́том, но и в друго́м слу́че, я не признаю́! Фана́тики – это стра́шны лю́ди. Гм. Это ни к чему́ соверше́н. Ма́ма, ма́ма, я да́же не по́мню, шоб она́… Ну, иногда́ там перекре́стится, «сла́ва Бо́гу», садя́тся за сто́л, когда́ выхо́дят из стола́, благодаря́т Го́спода. Но на мое́й па́мяти, я не зна́ю, шоб ма́ма куда́-то ходи́ла в це́рковь. Когда́ це́рковь была́ закры́та, щас я вам год не скажу́, я уже́ то́же ста́ла забыва́ть. По бли́зости це́ркви не́ было. И наско́лько я по́мню, на большо́й террито́рии вот одна́ Покро́вская ц́ерковь, она так и не закрыва́лась. Как её откры́ли в девятна́дцатом ве́ке, так она́ и до си́х пор, до сего́дняшнего дня существу́ет, и рабо́тат и принима́ет 

1.0

['Терентьева Надежда Ивановна']

'вечером перед сном молилась, утром вставала молилась. но она никогда не говорила: «внученька, встань со мной, давай». она много знала молитв, но я ни одной <смеётся>. это честно говорю. я человек православный, но не фанатично верующий. фанатизм я в любом случе не токо в этом, но и в другом случе, я не признаю! фанатики – это страшны люди. гм. это ни к чему совершен. мама, мама, я даже не помню, шоб она… ну, иногда там перекрестится, «слава богу», садятся за стол, когда выходят из стола, благодарят господа. но на моей памяти, я не знаю, шоб мама куда-то ходила в церковь. когда церковь была закрыта, щас я вам год не скажу, я уже тоже стала забывать. по близости церкви не было. и насколько я помню, на большой территории вот одна покровская церковь, она так и не закрывалась. как её открыли в девятнадцатом веке, так она и до сих пор, до сегодняшнего дня существует, и работат и принимает прихожан. где были ещё церкви? а потом, девочки милые, и не до церкви было. и не до обрядов этих было, п

0.69

...

['Караваева Евгения Изосимовна']

'не хожу. хожу в обшом ставить свечки, вот это, а мне не выстоять там, дак я поэтому с палочкой, дак я поэтому и болюсь. тут сходила, да мне худо там было что надо было скорую вплоть до того, что вызывать, я говорю, а зачем мне буду ходить, так дома, то что моё то. вера в бога есть, хоть нас учили в школе, что бога нет. было ведь только и разговоров – бога нет, и в церковь, в церковь тогда, конечно, и думы не было, и церкви не было. в деревне жили, вот, но верить всё равно так вот верю. как скажешь-то, вот, в церковь не хожу, но верю.\n\nа в кириллове здесь есь, в кириллове есь. в монастыре там церковь есь. большинство если, когда иду, дак я свечки ставить да это всё, дак которая в монастыре, а не та, которая вот новая тут, казанская та, в этой я, конечно, не бывала, а в ту хожу.\n\n крещёная. меня уже в песят лет крестили. сама крестилась. когда была маленька, у нас не было церкви. а во время войны, какая может быть церковь, да всё, так? и поэтому были мы не крещёные. дак я уже крести

0.61

['Киршина Нина Александровна']

'\nага. вон тоже, вон там все иконки. танюшка всё покупала, я много то не молюсь, подаю эти записки в церковь подаю, сама редко хожу, не могу, ноги не ходят дак. да этот вечерние то молитвы прочитаю, а так я всех обожаю, не то что я там уж неистово молюсь, но поразговариваю с иконками, поразговариваю. \n\nа иконы у меня: никола угодник вот в середине, тут это царица небесная, это порасковья, это порасковья, тут успенье божьей матери, это неупиваемая чаша.\n\nда, украшено скатерками украшены. на день то рождения, юбилей то был, всё перевешали, и тут то всё этот прибрано было, новые повешаны. это вот книжек столько у меня, книжек столько, библия. я тогда иногда читаю библию, но редко тоже, редко тоже читаю.\n\nа молитвы только вечерние, больше ничего. \n\nда, только вечерние.\n\nда, помню.\n\nнет. не ходила часто.\n\nя не помню как крестили меня, но я помню что на нашей улице, где мы жили, там жили монашки и там батюшко приходил. вот он крестил там ребятишек, и мы бегали вокруг купели вс